In [127]:
from PySide6 import QtGui 
import matplotlib
import matplotlib.pyplot as plt
import sys
import shutil
import os
import time

matplotlib.use('Qt5Agg')
from PySide6.QtWidgets import QMainWindow, QApplication
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from matplotlib.patches import Circle
from matplotlib import pyplot as plt
import numpy as np
from PySide6.QtWidgets import (QApplication, 
                               QWidget, 
                               QInputDialog, 
                               QLineEdit, 
                               QFileDialog, 
                               QPushButton, 
                               QVBoxLayout,
                               QHBoxLayout,
                               QLabel,
                               QGridLayout
                               )
import h5py

In [131]:
make_copy = False
class FigurePlot(QWidget):
    def __init__(self, parent=None, width=5, height=4, dpi=200):
        super(FigurePlot, self).__init__(parent)
        
        self.fig = Figure(figsize=(width,height), dpi=dpi)
        self.canvas = FigureCanvas(self.fig)
        self.axes = self.fig.add_subplot(111)
        
        self.make_copy = False
        self.path = None
        self.curr_idx = 0
        self.all_patterns = None
        self.len = 0
        self.output = ''
        
        self.ui()
        
    def ui(self, *args, **kwargs):
        # INSTRUCTIONS
        curr_file = QLabel("Please browse for a file to display\n\nNote, selecting 'Trash', will delete the \nimage you've currently selected. \nToggling 'Copy File' before browsing for a pattern\n will make a copy of the file.", self)
        curr_file.setStyleSheet(
            '''
            color: #E8E8E8;
            background-color: #2C2C2C;
            margin: 20px;
            padding: 20px;
            width: 100px;
            height: 120px;
            border: 3px solid #A0A0A0;
            border-radius: 10px;
            font-size: 13pt;
            '''
        )
        
        # TERMINAL
        self.terminal = QLabel(f">>{self.output}")
        self.terminal.setStyleSheet(
            '''
            color: #E8E8E8;
            background-color: #2C2C2C;
            margin: 20px;
            padding: 20px;
            width: 100px;
            height: 120px;
            border: 3px solid #A0A0A0;
            border-radius: 10px;
            font-size: 13pt;
            '''
        )
        
        text_layout = QVBoxLayout()
        text_layout.addWidget(curr_file)
        text_layout.addWidget(self.terminal)
        
        
        # TOGGLE 
        self.button0 = QPushButton('Copy File')
        self.button0.toggled.connect(self.copy_file)
        self.button0.setFixedSize(100, 30)
        self.button0.setCheckable(True)
        self.button0.setStyleSheet( ## green
            '''
            background: #2ed573;
            color: white;
            border: 1px solid white;
            border-radius: 5px;
            '''
        )
        
        toggle_layout = QGridLayout()
        toggle_layout.addWidget(self.button0, 0, 0)
        
        # BUTTONS
        self.button1 = QPushButton('Browse')
        self.button1.clicked.connect(self.open_file)
        self.button2 = QPushButton('Plot')
        self.button2.clicked.connect(self.init_graph)
        self.button3 = QPushButton('<-')
        self.button3.clicked.connect(lambda: self.next_graph(False))
        self.button4 = QPushButton('->')
        self.button4.clicked.connect(lambda: self.next_graph(True))
        self.button5 = QPushButton('Close App')
        self.button5.clicked.connect(self.close_mainwindow)
        self.button6 = QPushButton('Trash')
        self.button6.clicked.connect(self.delete_graph)
    
        buttons = [self.button1, self.button2, self.button3, self.button4, self.button5, self.button6]
        for button in buttons:
            button.setFixedSize(80, 30)
            button.setStyleSheet( ## blue
                '''
                background: #1e90ff;
                color: white;
                border: 1px solid white;
                border-radius: 5px;
                '''
            )

        button_layout = QGridLayout()
        button_layout.addWidget(self.button1, 0, 0)
        button_layout.addWidget(self.button2, 0, 1)
        button_layout.addWidget(self.button3, 1, 0)
        button_layout.addWidget(self.button4, 1, 1)
        button_layout.addWidget(self.button5, 2, 0)
        button_layout.addWidget(self.button6, 2, 1)
        self.button5.setStyleSheet(
            '''
            background: #ff4757;
            color: white;
            border: 1px solid white;
            border-radius: 5px;
            '''
        )
        self.button6.setStyleSheet( ## red
            '''
            background: #ff4757;
            color: white;
            border: 1px solid white;
            border-radius: 5px;
            '''
        )
        
        ## COMBINING UI LAYOUTS
        ui_layout =  QVBoxLayout()
        ui_layout.addLayout(text_layout)
        ui_layout.addLayout(toggle_layout)
        ui_layout.addLayout(button_layout)
        
        ## COMBINING UI WITH CANVAS
        main_layout = QHBoxLayout()
        main_layout.addWidget(self.canvas)
        main_layout.addLayout(ui_layout)

        self.setLayout(main_layout)
        # self.setLayout(button_layout)
        # self.setLayout

    def open_file(self, *args, **kwargs):
        self.print_terminal(f"Browsing")
        opts = QFileDialog.Options()
        opts |= QFileDialog.DontUseNativeDialog
        file, _ = QFileDialog.getOpenFileName(self,
            "QFileDialog.getOpenFileName()", 
            "","All Files (*);;Python Files (*.py)", 
            options=opts)
        self.path = file
        self.print_terminal(f"Opened file at: {os.path.relpath(self.path)}\n>> Click 'Plot' to view")
        if self.make_copy:
            
            version = 0
            
            trash_path = os.path.join(os.path.dirname(self.path), "trash")
            if not os.path.exists(trash_path):
                os.makedirs(trash_path)
            
            split_path = os.path.basename(self.path).split(".hp")[0]
            copy_path = os.path.join(trash_path, split_path +"_copy_0.hp")
            while os.path.exists(copy_path):
                version += 1
                copy_path = os.path.join(trash_path, split_path + f"_copy_{version}.hp")
            
            copy_path = os.path.join(trash_path, split_path + f"_copy_{version}.hp")
            self.print_terminal(f"Making a copy at: {os.path.relpath(copy_path)}")
            shutil.copy(file, copy_path)
        
    def copy_file(self):
        self.make_copy = not self.make_copy

    def init_graph(self, *args, **kwargs):
        if self.path:
            with h5py.File(self.path, 'a') as f:
                self.all_patterns = f['diffraction']['micrograph'][:]
                self.len = len(self.all_patterns)
                self.plot_graph()
                self.print_terminal(f"Plotted {os.path.relpath(self.path)}\n>> Number of patterns: {self.len}")            
    
    def plot_graph(self, *args, **kwargs):
        if self.len > 0:
            self.curr_pattern = self.all_patterns[self.curr_idx]
            self.axes.clear()
            self.axes.imshow(self.curr_pattern, cmap='gray')
            self.axes.set_title(f" Pattern {self.curr_idx + 1}/{self.len}")
            self.canvas.draw()
    
    def next_graph(self, next: bool = None, *args, **kwargs):
        if next is not None:
            if next:
                self.curr_idx += 1
                if self.curr_idx == self.len:
                    self.curr_idx = 0
                self.print_terminal("1 forward")
            
            else:
                self.curr_idx -= 1
                if self.curr_idx == -1:
                    self.curr_idx = self.len - 1
                self.print_terminal("1 back")
            self.plot_graph()
    
    def close_mainwindow(self):
        main_window = self.window()
        if isinstance(main_window, QMainWindow):
            main_window.close()
    
    def delete_graph(self, *args, **kwargs):
        if self.len > 0:
            with h5py.File(self.path, 'a') as f:
                patterns = f['diffraction']['micrograph'][:]
                deleted_index = self.curr_idx
                new_data = np.delete(patterns, self.curr_idx, axis=0)
                del f['diffraction']['micrograph']
                f.create_dataset('diffraction/micrograph', data=new_data)
                self.all_patterns = new_data
                self.len = len(self.all_patterns)
                if self.curr_idx >= self.len:
                    self.curr_idx = 0
                self.output = f">> Deleted image at index {deleted_index+1}"
                print(self.output)
                self.terminal.setText(self.output)
                self.plot_graph()
                
    def print_terminal(self, message):
            self.output = f">> {message}"
            print(self.output)
            time.sleep(0.05)
            self.terminal.setText(">>")
            time.sleep(0.05)
            self.terminal.setText(self.output)
        
class MainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        self.title = "Plot"
        self.setWindowTitle(self.title)
        self.plot = FigurePlot(self, width=15, height =15, dpi=200)
        self.setCentralWidget(self.plot)

        
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

w = MainWindow()
w.show()
app.exec()

>> Browsing
>> Opened file at: patterns/1.hp
>> Click 'Plot' to view
>> Plotting patterns/1.hp
>> Number of patterns: 7
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> Deleted image at index 2
>> Browsing
>> Opened file at: patterns/0.hp
>> Click 'Plot' to view
>> Making a copy at: /Users/jonathanchoi/Desktop/GitHub Projects/crystal_GUI/patterns/trash/0_copy_1.hp
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> Browsing
>> Opened file at: patterns/0.hp
>> Click 'Plot' to view
>> Making a copy at: /Users/jonathanchoi/Desktop/GitHub Projects/crystal_GUI/patterns/trash/0_copy_2.hp
>> Plotting patterns/0.hp
>> Number of patterns: 1


IndexError: index 4 is out of bounds for axis 0 with size 1

>> Browsing
>> Opened file at: patterns/1.hp
>> Click 'Plot' to view
>> Plotting patterns/1.hp
>> Number of patterns: 6
>> 1 forward
>> 1 forward
>> 1 forward
>> 1 forward
>> Deleted image at index 2
>> Deleted image at index 2
>> Deleted image at index 2
>> Deleted image at index 2
>> Deleted image at index 0
>> Browsing
>> Opened file at: patterns/0.hp
>> Click 'Plot' to view
>> Browsing
>> Opened file at: patterns/1.hp
>> Click 'Plot' to view
>> Plotting patterns/1.hp
>> Number of patterns: 1
>> Plotting patterns/1.hp
>> Number of patterns: 1
>> 1 forward
>> 1 forward
>> 1 forward


0

In [50]:
h = True

h = not h
print(h)

False


In [68]:
os.path.dirname("data/0.hp")

if not os.path.exists(os.path.join(os.path.dirname("data_pattern/0.hp"), "trash")):
    print("true")
    os.makedirs(os.path.join(os.path.dirname("data_pattern/0.hp"), "trash"))



true


In [59]:
print(os.path.join("trash/", os.path.dirname("data/0.hp")))

trash/data


In [72]:
os.path.relpath("/Users/jonathanchoi/Desktop/GitHub Projects/crystal_GUI/README.md")

'README.md'

In [77]:


os.path.basename("/Users/jonathanchoi/Desktop/GitHub Projects/crystal_GUI/README.md")

'README.md'